# OpenRLHF

| Feature | OpenRLHF | DSChat | CAIChat | TRL |
| ------------- |:-------------:| :-------------:| :-------------:| :-------------:|
| 70B+ Full Tuning with 16 A100-80GB      | ✅ | ❌ | ❌ | ❌ |
| 7B Full Tuning with 4 RTX4090 | ✅      |    ❌ | ❌ | ❌ |
| 34B DPO Full Tuning with 8 A100-80GB | ✅      |    ❌ | ❌ | ❌ |  
| Inference Engine in PPO | ✅      |    ✅ | ❌ | ❌ |  
| PPO Implementation Tricks | ✅      |    ❌ | ❌ | ✅ |
| Support QLoRA | ✅      |    ❌ | ❌ | ✅ | 
| Support Mixtral 8*7b | ✅      |    ❌ | ❌ | ❌ |  
| Support Unmerged Actor-Critic | ✅     |   ✅ | ✅ | ❌ | 
| Support Multiple Reward Models | ✅      |    ❌ | ❌ | ❌ |   
| Support Huggingface Models | ✅      |    ✅ | ✅ | ✅ | 
| Easy-to-use | ✅      |   ❌ (HybridEngine bugs) | ✅ | ✅ | 

## PPO implementation detail

`train_ppo_llama.sh`:

```shell
openrlhf.cli.train_ppo \
   --pretrain OpenRLHF/Llama-3-8b-sft-mixture \
   --reward_pretrain OpenRLHF/Llama-3-8b-rm-mixture \
   --save_path ./checkpoint/llama-3-8b-rlhf \
   --save_steps -1 \
   --logging_steps 1 \
   --eval_steps -1 \
   --micro_train_batch_size 2 \
   --train_batch_size 128 \
   --micro_rollout_batch_size 4 \
   --rollout_batch_size 1024 \
   --max_epochs 1 \
   --prompt_max_len 1024 \
   --generate_max_len 1024 \
   --zero_stage 2 \
   --bf16 \
   --actor_learning_rate 5e-7 \
   --critic_learning_rate 9e-6 \
   --init_kl_coef 0.01 \
   --prompt_data OpenRLHF/prompt-collection-v0.1 \
   --input_key context_messages \
   --apply_chat_template \
   --max_samples 100000 \
   --normalize_reward \
   --adam_offload \
   --flash_attn \
   --load_checkpoint \
   --gradient_checkpointing
```

`trainer.fit`:

```python
# The outermost layer
for episode in range(args.num_episodes):
    
    # len(rand_prompts) == args.rollout_batch_size // strategy.world_size
    # drop_last=True
    for rand_prompts in self.prompts_dataloader:
        
        # `make_experience_list` make a list of experience with the micro_rollout_batch_size.
        # `append` first split_experience_batch, then extend items
        # micro_rollout_batch_size makes no real difference
        for experience in self.experience_maker.make_experience_list(rand_prompts, **self.generate_kwargs):
            self.replay_buffer.append(experience)

        self.replay_buffer.normalize("advantages", self.strategy)
        status = self.ppo_train(steps)
        self.replay_buffer.clear()
```

`ppo_train`:

```python
for epoch in range(self.max_epochs):
    
    # go over replay buffer
    # len(experience) == micro_train_batch_size
    # train_batch_size makes no real difference
    for experience in pbar:

        # training step
        status = {}
        status = self.training_step_actor(experience)
        status.update(self.training_step_critic(experience))
```